In [1]:
import numpy as np

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from stardist import random_label_cmap
from stardist.matching import matching_dataset
from stardist.models import Config2D, StarDist2D

from sklearn.model_selection import KFold
import pandas as pd
import gc
import tensorflow as tf

np.random.seed(42)
lbl_cmap = random_label_cmap()

2024-07-12 20:09:15.340006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
X = sorted(glob('datasets/not_split/Aggregates/images/*.tif'))
Y = sorted(glob('datasets/not_split/Aggregates/masks/*.tif'))

In [3]:
assert all(Path(x).name==Path(y).name for x,y in zip(X,Y))

X = list(map(imread,X))
Y = list(map(imread,Y))

In [4]:
X = [normalize(x,1,99.9,axis=(0,1)) for x in tqdm(X)]

100%|██████████| 1284/1284 [00:00<00:00, 1326.42it/s]


In [5]:
model_name = 'aggregates_pretrained'
model_dir = 'final_models'

results_file = 'results/results.csv'

In [6]:
conf = Config2D (
    n_rays       = 32,
    grid         = (2,2),
    n_channel_in = 1,
    train_dist_loss = 'iou',
    train_epochs = 350
)

vars(conf)

{'n_dim': 2,
 'axes': 'YXC',
 'n_channel_in': 1,
 'n_channel_out': 33,
 'train_checkpoint': 'weights_best.h5',
 'train_checkpoint_last': 'weights_last.h5',
 'train_checkpoint_epoch': 'weights_now.h5',
 'n_rays': 32,
 'grid': (2, 2),
 'backbone': 'unet',
 'n_classes': None,
 'unet_n_depth': 3,
 'unet_kernel_size': (3, 3),
 'unet_n_filter_base': 32,
 'unet_n_conv_per_depth': 2,
 'unet_pool': (2, 2),
 'unet_activation': 'relu',
 'unet_last_activation': 'relu',
 'unet_batch_norm': False,
 'unet_dropout': 0.0,
 'unet_prefix': '',
 'net_conv_after_unet': 128,
 'net_input_shape': (None, None, 1),
 'net_mask_shape': (None, None, 1),
 'train_shape_completion': False,
 'train_completion_crop': 32,
 'train_patch_size': (256, 256),
 'train_background_reg': 0.0001,
 'train_foreground_only': 0.9,
 'train_sample_cache': True,
 'train_dist_loss': 'iou',
 'train_loss_weights': (1, 0.2),
 'train_class_weights': (1, 1),
 'train_epochs': 350,
 'train_steps_per_epoch': 100,
 'train_learning_rate': 0.0003,


In [7]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img


def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    # x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

In [8]:
def cross_validate_stardist(images, masks, n_splits=5, start_from_fold=0):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(images)):

        if fold < start_from_fold:
            continue  # Skip to the desired starting fold

        print(f"Starting fold {fold + 1}/{n_splits}...")
        
        # Splitting the dataset
        train_images, test_images = [images[i] for i in train_idx], [images[i] for i in test_idx]
        train_masks, test_masks = [masks[i] for i in train_idx], [masks[i] for i in test_idx]

        # model = StarDist2D.from_pretrained('2D_versatile_fluo')

        # Initialize Stardist model
        model = StarDist2D(conf, name=model_name+str(fold), basedir=model_dir)

        model.train(train_images, train_masks, validation_data=(test_images,test_masks), augmenter=augmenter, seed=42)
        model.optimize_thresholds(test_images, test_masks)
        
        # Evaluation
        Y_pred = [model.predict_instances(x, show_tile_progress=False)[0] for x in test_images]

        stat = matching_dataset(test_masks, Y_pred, thresh=0.1, show_progress=False)

        data = {
            'model': [model_name+str(fold)],
            'tp': [stat.tp],
            'fp': [stat.fp],
            'fn': [stat.fn],
            'precision': [stat.precision],
            'recall': [stat.recall],
            'accuracy': [stat.accuracy]
        }

        df = pd.DataFrame(data)

        df1 = pd.read_csv(results_file)

        merged_df = pd.concat([df1, df], ignore_index=True)
        merged_df = merged_df.sort_values(by='model', ascending=True)

        merged_df.to_csv(results_file, index=False)
        
        print(f"Fold {fold + 1} completed.")


In [9]:
cross_validate_stardist(X, Y, n_splits=5, start_from_fold=4)

Starting fold 5/5...


base_model.py (149): output path for model already exists, files may be overwritten: /home/deivis/VU/SRP/code/stardist/final_v1/final_models/aggregates_pretrained4
2024-07-12 20:09:25.878794: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-07-12 20:09:25.881756: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-07-12 20:09:26.679394: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-12 20:09:26.679448: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4070 Ti computeCapability: 8.9
coreClock: 2.61GHz coreCount: 60 deviceMemorySize: 11.99GiB deviceMemoryBandwidth: 469.43GiB/s
2024-07-12 20:09:26.679486: I tensorflow/s

Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.417819, nms_thresh=0.5.


2024-07-12 20:09:28.838415: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-07-12 20:09:28.838793: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4192045000 Hz


Epoch 1/350
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function _gcd_import at 0x7f5f052a2430>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function _gcd_import at 0x7f5f052a2430>. Note that functions defined in certain environments, like the interactive 

2024-07-12 20:09:30.056308: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2024-07-12 20:09:32.182662: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2024-07-12 20:09:32.246190: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-07-12 20:09:34.135257: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-07-12 20:09:34.743674: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


100/100 [==============================] - 92s 33ms/step - loss: 0.3944 - prob_loss: 0.2091 - dist_loss: 0.9269 - prob_kld: 0.2007 - dist_relevant_mae: 4.1387 - dist_relevant_mse: 31.2399 - dist_dist_iou_metric: 0.1238 - val_loss: 0.1430 - val_prob_loss: 0.0115 - val_dist_loss: 0.6575 - val_prob_kld: 0.0105 - val_dist_relevant_mae: 2.7350 - val_dist_relevant_mse: 12.4098 - val_dist_dist_iou_metric: 0.3433
Epoch 2/350
100/100 [==============================] - 3s 26ms/step - loss: 0.1401 - prob_loss: 0.0108 - dist_loss: 0.6464 - prob_kld: 0.0099 - dist_relevant_mae: 2.4879 - dist_relevant_mse: 9.7182 - dist_dist_iou_metric: 0.3488 - val_loss: 0.1357 - val_prob_loss: 0.0103 - val_dist_loss: 0.6274 - val_prob_kld: 0.0093 - val_dist_relevant_mae: 2.7067 - val_dist_relevant_mse: 11.1631 - val_dist_dist_iou_metric: 0.3729
Epoch 3/350
100/100 [==============================] - 3s 26ms/step - loss: 0.1354 - prob_loss: 0.0113 - dist_loss: 0.6205 - prob_kld: 0.0103 - dist_relevant_mae: 2.3946 - 

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:08<00:02,  1.71it/s, 0.558 -> 0.618]


Using optimized values: prob_thresh=0.559911, nms_thresh=0.3.
Saving to 'thresholds.json'.
Fold 5 completed.


In [13]:
model = StarDist2D(conf, name='yh2ax0', basedir=model_dir)
Y_pred = [model.predict_instances(x, show_tile_progress=False)[0] for x in X]

stat = matching_dataset(Y, Y_pred, thresh=0.1, show_progress=False)

data = {
    'model': ['testasm'],
    'tp': [stat.tp],
    'fp': [stat.fp],
    'fn': [stat.fn],
    'precision': [stat.precision],
    'recall': [stat.recall],
    'accuracy': [stat.accuracy]
}

df = pd.DataFrame(data)

df1 = pd.read_csv(results_file)

merged_df = pd.concat([df1, df], ignore_index=True)
merged_df = merged_df.sort_values(by='model', ascending=True)

merged_df.to_csv(results_file, index=False)

Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.623276, nms_thresh=0.5.
